# Introducing the Keras Functional API

**Learning Objectives**
  - Understand embeddings and how to create them with the feature column API
  - Understand Deep and Wide models and when to use them
  - Understand the Keras functional API and how to build a deep and wide model with it
  - Learn how to train a Keras model at scale on GCP

## Introduction

In the last notebook, we learned about the Keras Sequential API. The [Keras Functional API](https://www.tensorflow.org/guide/keras#functional_api) provides an alternate way of building models which is more flexible. With the Functional API, we can build models with more complex topologies, multiple input or output layers, shared layers or non-sequential data flows (e.g. residual layers).

In this notebook we'll use what we learned about feature columns to build a Wide & Deep model. Recall, that the idea behind Wide & Deep models is to join the two methods of learning through memorization and generalization by making a wide linear model and a deep learning model to accommodate both. 

<img src='assets/wide_deep.png' width='80%'>
<sup>(image: https://ai.googleblog.com/2016/06/wide-deep-learning-better-together-with.html)</sup>

The Wide part of the model is associated with the memory element. In this case, we train a linear model with a wide set of crossed features and learn the correlation of this related data with the assigned label. The Deep part of the model is associated with the generalization element where we use embedding vectors for features. The best embeddings are then learned through the training process. While both of these methods can work well alone, Wide & Deep models excel by combining these techniques together. 

Once we have trained our model, we will see how to train our model at scale on GCP using AI Platform.

In [1]:
#  Ensure that we have the latest version of Tensorflow installed.
!pip3 freeze | grep tf-nightly-2.0-preview || pip3 install tf-nightly-2.0-preview

tf-nightly-2.0-preview==2.0.0.dev20190919


Start by importing the necessary libraries for this lab.

In [2]:
import datetime
import os
import shutil

import numpy as np
import pandas as pd
import tensorflow as tf

%matplotlib inline
from matplotlib import pyplot as plt
from tensorflow import keras

from tensorflow import feature_column as fc

from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.callbacks import TensorBoard

print(tf.__version__)

2.0.0-dev20190919


## Load raw data 

We will use the taxifare dataset, using the CSV files that we created in the first notebook of this sequence. Those files have been saved into `../data`.

In [3]:
!ls -l ../data/*.csv

-rw-r--r--  1 munn  primarygroup  123590 Sep 19 18:08 ../data/taxi-test.csv
-rw-r--r--  1 munn  primarygroup  579055 Sep 19 18:08 ../data/taxi-train.csv
-rw-r--r--  1 munn  primarygroup  123114 Sep 19 18:08 ../data/taxi-valid.csv


## Use tf.data to read the CSV files

We wrote these functions for reading data from the csv files above in the [previous notebook](2_dataset_api.ipynb). For this lab we will also include some additional engineered features in our model. In particular, we will compute the difference in latitude and longitude, as well as the Euclidean distance between the pick-up and drop-off locations. We can accomplish this by adding these new features to the features dictionary with the function `add_engineered_features` below. 

Note that we include a call to this function when collecting our features dict and labels in the `features_and_labels` function below as well. 

In [4]:
CSV_COLUMNS = [
    'fare_amount',
    'pickup_datetime',
    'pickup_longitude',
    'pickup_latitude',
    'dropoff_longitude',
    'dropoff_latitude',
    'passenger_count',
    'key'
]
LABEL_COLUMN = 'fare_amount'
DEFAULTS = [[0.0], ['na'], [0.0], [0.0], [0.0], [0.0], [0.0], ['na']]
UNWANTED_COLS = ['pickup_datetime', 'key']

def add_engineered_features(features):
    # Compute Euclidean distance
    features["latdiff"] = features["pickup_latitude"] - features["dropoff_latitude"]
    features["londiff"] = features["pickup_longitude"] - features["dropoff_longitude"]
    features["euclidean_dist"] = tf.sqrt(
        x=features["latdiff"]**2 + features["londiff"]**2)

    return features


def features_and_labels(row_data):
    label = row_data.pop(LABEL_COLUMN)
    features = row_data
    
    # Add engineered features
    features = add_engineered_features(features)
    
    for unwanted_col in UNWANTED_COLS:
        features.pop(unwanted_col)

    return features, label


def create_dataset(pattern, batch_size=1, mode=tf.estimator.ModeKeys.EVAL):
    dataset = tf.data.experimental.make_csv_dataset(
        pattern, batch_size, CSV_COLUMNS, DEFAULTS)

    dataset = dataset.map(features_and_labels)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        dataset = dataset.shuffle(buffer_size=1000).repeat()

    # take advantage of multi-threading; 1=AUTOTUNE
    dataset = dataset.prefetch(1)
    return dataset

## Feature columns for Wide and Deep model

For the Wide columns, we will create feature columns of crossed features. To do this, we'll create a collection of Tensorflow feature columns to pass to the `tf.feature_column.crossed_column` constructor. The Deep columns will consist of numeric columns and the embedding columns we want to create. 

In [5]:
# 1. Bucketize latitudes and longitudes
NBUCKETS = 16
latbuckets = np.linspace(start=38.0, stop=42.0, num=NBUCKETS).tolist()
lonbuckets = np.linspace(start=-76.0, stop=-72.0, num=NBUCKETS).tolist()

fc_bucketized_plat = fc.bucketized_column(
    source_column=fc.numeric_column(key="pickup_longitude"), boundaries=lonbuckets)
fc_bucketized_plon = fc.bucketized_column(
    source_column=fc.numeric_column(key="pickup_latitude"), boundaries=latbuckets)
fc_bucketized_dlat = fc.bucketized_column(
    source_column=fc.numeric_column(key="dropoff_longitude"), boundaries=lonbuckets)
fc_bucketized_dlon = fc.bucketized_column(
    source_column=fc.numeric_column(key="dropoff_latitude"), boundaries=latbuckets)

# 2. Cross features for locations
fc_crossed_dloc = fc.crossed_column(
    keys=[fc_bucketized_dlat, fc_bucketized_dlon],
    hash_bucket_size=NBUCKETS * NBUCKETS)
fc_crossed_ploc = fc.crossed_column(
    keys=[fc_bucketized_plat, fc_bucketized_plon],
    hash_bucket_size=NBUCKETS * NBUCKETS)
fc_crossed_pd_pair = fc.crossed_column(
    keys=[fc_crossed_dloc, fc_crossed_ploc],
    hash_bucket_size=NBUCKETS**4)

# 3. Create embedding columns for the crossed columns
fc_pd_pair = fc.embedding_column(categorical_column=fc_crossed_pd_pair, dimension=3)
fc_dloc = fc.embedding_column(categorical_column=fc_crossed_dloc, dimension=3)
fc_ploc = fc.embedding_column(categorical_column=fc_crossed_ploc, dimension=3)

### Gather list of feature columns

Next we gather the list of wide and deep feature columns we'll pass to our Wide & Deep model in Tensorflow. Recall, wide columns are sparse, have linear relationship with the output while continuous columns are deep, have a complex relationship with the output. We will use our previously bucketized columns to collect crossed feature columns and sparse feature columns for our wide columns, and embedding feature columns and numeric features columns for the deep columns.

In [6]:
wide_columns = [
    # One-hot encoded feature crosses
    fc.indicator_column(fc_crossed_dloc),
    fc.indicator_column(fc_crossed_ploc),
    fc.indicator_column(fc_crossed_pd_pair)
]

deep_columns = [
    # Embedding_column to "group" together ...
    fc.embedding_column(categorical_column=fc_crossed_pd_pair, dimension=10),

    # Numeric columns
    fc.numeric_column(key="pickup_latitude"),
    fc.numeric_column(key="pickup_longitude"),
    fc.numeric_column(key="dropoff_longitude"),
    fc.numeric_column(key="dropoff_latitude"),
    fc.numeric_column(key="latdiff"),
    fc.numeric_column(key="londiff"),
    fc.numeric_column(key="euclidean_dist"),
]

## Build a Wide and Deep model in Keras

To build a wide-and-deep network, we connect the sparse (i.e. wide) features directly to the output node, but pass the dense (i.e. deep) features through a set of fully connected layers. Here’s that model architecture looks using the Functional API.

In [7]:
INPUT_COLS = [
    'pickup_longitude',
    'pickup_latitude',
    'dropoff_longitude',
    'dropoff_latitude',
    'passenger_count',
    'latdiff',
    'londiff',
    'euclidean_dist'
]

inputs = {colname : tf.keras.layers.Input(name=colname, shape=(), dtype='float32')
          for colname in INPUT_COLS
}

In [8]:
inputs

{'pickup_longitude': <tf.Tensor 'pickup_longitude:0' shape=(None,) dtype=float32>,
 'pickup_latitude': <tf.Tensor 'pickup_latitude:0' shape=(None,) dtype=float32>,
 'dropoff_longitude': <tf.Tensor 'dropoff_longitude:0' shape=(None,) dtype=float32>,
 'dropoff_latitude': <tf.Tensor 'dropoff_latitude:0' shape=(None,) dtype=float32>,
 'passenger_count': <tf.Tensor 'passenger_count:0' shape=(None,) dtype=float32>,
 'latdiff': <tf.Tensor 'latdiff:0' shape=(None,) dtype=float32>,
 'londiff': <tf.Tensor 'londiff:0' shape=(None,) dtype=float32>,
 'euclidean_dist': <tf.Tensor 'euclidean_dist:0' shape=(None,) dtype=float32>}

In [9]:
# Create the deep part of model
deep = tf.keras.layers.DenseFeatures(deep_columns, name='deep_inputs')(inputs)

dnn_hidden_units = [10,5]
for numnodes in dnn_hidden_units:
    deep = tf.keras.layers.Dense(numnodes, activation='relu')(deep) 

# Create the wide part of model
wide = tf.keras.layers.DenseFeatures(wide_columns, name='wide_inputs')(inputs)

# Combine deep and wide parts of the model
combined = tf.keras.layers.concatenate(inputs=[deep, wide], name='combined')

# Map the combined outputs into a single prediction value
output = tf.keras.layers.Dense(units=1, activation=None, name='prediction')(combined)


Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


In [10]:
# Finalize the model
model = tf.keras.Model(inputs=list(inputs.values()), outputs=output)

In [20]:
tf.keras.utils.plot_model(model, show_shapes=False, rankdir='LR')

Exception: "dot" not found in path.

In [16]:
TRAIN_BATCH_SIZE = 1000
NUM_TRAIN_EXAMPLES = 10000 * 5  # training dataset will repeat, wrap around
NUM_EVALS = 50  # how many times to evaluate
NUM_EVAL_EXAMPLES = 10000  # enough to get a reasonable sample

trainds = create_dataset(
    pattern='../data/taxi-train*',
    batch_size=TRAIN_BATCH_SIZE,
    mode=tf.estimator.ModeKeys.TRAIN)

evalds = create_dataset(
    pattern='../data/taxi-valid*',
    batch_size=1000,
    mode=tf.estimator.ModeKeys.EVAL).take(NUM_EVAL_EXAMPLES//1000)

Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_deterministic`.
Instructions for updating:
Use `tf.data.Dataset.shuffle(buffer_size, seed)` followed by `tf.data.Dataset.repeat(count)`. Static tf.data optimizations will take care of using the fused implementation.


In [17]:
# Create a custom evalution metric
def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true)))


# Compile the keras model
model.compile(optimizer="adam", loss="mse", metrics=[rmse, "mse"])

In [19]:
%%time
steps_per_epoch = NUM_TRAIN_EXAMPLES // (TRAIN_BATCH_SIZE * NUM_EVALS)

OUTDIR = "./taxi_trained"
shutil.rmtree(path=OUTDIR, ignore_errors=True) # start fresh each time
history = model.fit(x=trainds,
                    steps_per_epoch=steps_per_epoch,
                    epochs=NUM_EVALS,
                    validation_data=evalds,
                    callbacks=[TensorBoard(OUTDIR)])

Train for 1 steps, validate for 10 steps
Epoch 1/50
1/1 [==============================] - 15s 15s/step - loss: 220.3724 - rmse: 14.8449 - mse: 220.3724 - val_loss: 247.9802 - val_rmse: 15.7295 - val_mse: 247.9803
Epoch 2/50
1/1 [==============================] - 4s 4s/step - loss: 242.5711 - rmse: 15.5747 - mse: 242.5711 - val_loss: 247.8867 - val_rmse: 15.7266 - val_mse: 247.8867
Epoch 3/50
1/1 [==============================] - 4s 4s/step - loss: 227.5333 - rmse: 15.0842 - mse: 227.5333 - val_loss: 247.7934 - val_rmse: 15.7236 - val_mse: 247.7934
Epoch 4/50
1/1 [==============================] - 4s 4s/step - loss: 266.0244 - rmse: 16.3103 - mse: 266.0244 - val_loss: 247.6998 - val_rmse: 15.7206 - val_mse: 247.6998
Epoch 5/50
1/1 [==============================] - 4s 4s/step - loss: 224.9883 - rmse: 14.9996 - mse: 224.9883 - val_loss: 247.6064 - val_rmse: 15.7176 - val_mse: 247.6064
Epoch 6/50
1/1 [==============================] - 3s 3s/step - loss: 246.0034 - rmse: 15.6845 - mse: 2

Epoch 49/50
1/1 [==============================] - 4s 4s/step - loss: 219.9075 - rmse: 14.8293 - mse: 219.9075 - val_loss: 243.5587 - val_rmse: 15.5882 - val_mse: 243.5587
Epoch 50/50
1/1 [==============================] - 3s 3s/step - loss: 197.3156 - rmse: 14.0469 - mse: 197.3156 - val_loss: 243.4683 - val_rmse: 15.5853 - val_mse: 243.4683
CPU times: user 8min 2s, sys: 7min 42s, total: 15min 45s
Wall time: 3min 4s
